In [44]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [49]:
hash("maaz")

4672656836405334870

In [51]:
hash("aaam")

8554639782290908677

In [47]:
hash("zaam")

2522791423183749923

In [5]:
imdb,info =tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [6]:
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [7]:
for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

In [8]:
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [9]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [10]:
# Hyperparameters on top for easy modification.
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
tokenizer =Tokenizer(num_words=vocab_size,oov_token=oov_tok)

In [14]:

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index


In [15]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [16]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [18]:
num_epochs = 5
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/5
782/782 [==============================] - 5s 5ms/step - loss: 0.6106 - accuracy: 0.6299 - val_loss: 0.3537 - val_accuracy: 0.8445
Epoch 2/5
782/782 [==============================] - 4s 5ms/step - loss: 0.2444 - accuracy: 0.9077 - val_loss: 0.3806 - val_accuracy: 0.8336
Epoch 3/5
782/782 [==============================] - 4s 5ms/step - loss: 0.0948 - accuracy: 0.9789 - val_loss: 0.4441 - val_accuracy: 0.8306
Epoch 4/5
782/782 [==============================] - 3s 4ms/step - loss: 0.0238 - accuracy: 0.9973 - val_loss: 0.5377 - val_accuracy: 0.8235
Epoch 5/5
782/782 [==============================] - 4s 4ms/step - loss: 0.0053 - accuracy: 0.9998 - val_loss: 0.6211 - val_accuracy: 0.8203


In [36]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


# Average Pooling 

In [41]:
Avgmodel = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
Avgmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
Avgmodel.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [42]:
num_epochs = 10
Avgmodel.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.5611 - accuracy: 0.7255 - val_loss: 0.3347 - val_accuracy: 0.8576
Epoch 2/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.2656 - accuracy: 0.8967 - val_loss: 0.3406 - val_accuracy: 0.8546
Epoch 3/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.2043 - accuracy: 0.9264 - val_loss: 0.3897 - val_accuracy: 0.8424
Epoch 4/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.1616 - accuracy: 0.9449 - val_loss: 0.4567 - val_accuracy: 0.8319
Epoch 5/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.1354 - accuracy: 0.9570 - val_loss: 0.5282 - val_accuracy: 0.8247
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.1082 - accuracy: 0.9671 - val_loss: 0.5863 - val_accuracy: 0.8192
Epoch 7/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.0903 - accuracy: 0.9753 - val_loss: 0.6915 - val_accuracy:

In [19]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [44]:
e1 = Avgmodel.layers[0]
weights = e1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [20]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')


In [27]:
len(reverse_word_index)

88583

In [28]:
weights.shape

(10000, 16)

In [30]:

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    print(word)
    
    embeddings = weights[word_num]
    print(embeddings.shape)
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

<OOV>
(16,)
the
(16,)
and
(16,)
a
(16,)
of
(16,)
to
(16,)
is
(16,)
br
(16,)
in
(16,)
it
(16,)
i
(16,)
this
(16,)
that
(16,)
was
(16,)
as
(16,)
for
(16,)
with
(16,)
movie
(16,)
but
(16,)
film
(16,)
on
(16,)
not
(16,)
you
(16,)
are
(16,)
his
(16,)
have
(16,)
he
(16,)
be
(16,)
one
(16,)
all
(16,)
at
(16,)
by
(16,)
an
(16,)
they
(16,)
who
(16,)
so
(16,)
from
(16,)
like
(16,)
her
(16,)
or
(16,)
just
(16,)
about
(16,)
it's
(16,)
out
(16,)
if
(16,)
has
(16,)
some
(16,)
there
(16,)
what
(16,)
good
(16,)
more
(16,)
when
(16,)
very
(16,)
up
(16,)
no
(16,)
time
(16,)
she
(16,)
even
(16,)
my
(16,)
would
(16,)
which
(16,)
only
(16,)
story
(16,)
really
(16,)
see
(16,)
their
(16,)
had
(16,)
can
(16,)
were
(16,)
me
(16,)
well
(16,)
than
(16,)
we
(16,)
much
(16,)
been
(16,)
bad
(16,)
get
(16,)
will
(16,)
do
(16,)
also
(16,)
into
(16,)
people
(16,)
other
(16,)
first
(16,)
great
(16,)
because
(16,)
how
(16,)
him
(16,)
most
(16,)
don't
(16,)
made
(16,)
its
(16,)
then
(16,)
way
(16,)
make
(16,)
them
(16,)


four
(16,)
songs
(16,)
basically
(16,)
attention
(16,)
7
(16,)
knows
(16,)
clearly
(16,)
supporting
(16,)
knew
(16,)
comic
(16,)
non
(16,)
british
(16,)
television
(16,)
fast
(16,)
earth
(16,)
country
(16,)
cheap
(16,)
class
(16,)
future
(16,)
silly
(16,)
thriller
(16,)
8
(16,)
king
(16,)
problems
(16,)
aren't
(16,)
easily
(16,)
words
(16,)
tells
(16,)
miss
(16,)
jack
(16,)
local
(16,)
sequence
(16,)
bring
(16,)
entertainment
(16,)
paul
(16,)
beyond
(16,)
upon
(16,)
whether
(16,)
predictable
(16,)
moving
(16,)
straight
(16,)
sets
(16,)
similar
(16,)
romantic
(16,)
review
(16,)
oscar
(16,)
falls
(16,)
mystery
(16,)
enjoyable
(16,)
needs
(16,)
rock
(16,)
appears
(16,)
talk
(16,)
george
(16,)
giving
(16,)
eye
(16,)
within
(16,)
richard
(16,)
ten
(16,)
animation
(16,)
message
(16,)
near
(16,)
theater
(16,)
above
(16,)
dull
(16,)
nearly
(16,)
sequel
(16,)
theme
(16,)
points
(16,)
'
(16,)
stand
(16,)
mention
(16,)
lady
(16,)
bunch
(16,)
add
(16,)
feels
(16,)
herself
(16,)
release
(16,)
red
(

falling
(16,)
lived
(16,)
hotel
(16,)
tired
(16,)
puts
(16,)
zero
(16,)
image
(16,)
pain
(16,)
lover
(16,)
everybody
(16,)
giant
(16,)
spot
(16,)
shock
(16,)
offer
(16,)
j
(16,)
suggest
(16,)
henry
(16,)
trailer
(16,)
include
(16,)
student
(16,)
adults
(16,)
confused
(16,)
difference
(16,)
fresh
(16,)
bruce
(16,)
followed
(16,)
kelly
(16,)
r
(16,)
appeared
(16,)
hasn't
(16,)
approach
(16,)
victims
(16,)
fellow
(16,)
christian
(16,)
gorgeous
(16,)
step
(16,)
hurt
(16,)
impact
(16,)
putting
(16,)
event
(16,)
sub
(16,)
mix
(16,)
murders
(16,)
share
(16,)
notice
(16,)
content
(16,)
confusing
(16,)
laughed
(16,)
11
(16,)
mediocre
(16,)
lacks
(16,)
summer
(16,)
supposedly
(16,)
direct
(16,)
actresses
(16,)
porn
(16,)
system
(16,)
page
(16,)
flaws
(16,)
moral
(16,)
wall
(16,)
holes
(16,)
billy
(16,)
worthy
(16,)
jerry
(16,)
creative
(16,)
tragedy
(16,)
rape
(16,)
relationships
(16,)
thin
(16,)
answer
(16,)
lighting
(16,)
race
(16,)
helps
(16,)
random
(16,)
ii
(16,)
gem
(16,)
jones
(16,)
merel

proper
(16,)
sympathetic
(16,)
warm
(16,)
individual
(16,)
rules
(16,)
drew
(16,)
learns
(16,)
embarrassing
(16,)
unrealistic
(16,)
portraying
(16,)
canadian
(16,)
safe
(16,)
victor
(16,)
appealing
(16,)
daniel
(16,)
dubbed
(16,)
marry
(16,)
depressing
(16,)
par
(16,)
chick
(16,)
recall
(16,)
shakespeare
(16,)
winner
(16,)
involves
(16,)
uk
(16,)
sequels
(16,)
hearted
(16,)
contrast
(16,)
freddy
(16,)
k
(16,)
chief
(16,)
matters
(16,)
correct
(16,)
haunting
(16,)
costume
(16,)
crowd
(16,)
woody
(16,)
paper
(16,)
claim
(16,)
vote
(16,)
strongly
(16,)
grow
(16,)
heck
(16,)
nominated
(16,)
research
(16,)
clue
(16,)
appropriate
(16,)
disgusting
(16,)
matt
(16,)
eight
(16,)
rose
(16,)
evidence
(16,)
facts
(16,)
protagonist
(16,)
joseph
(16,)
germany
(16,)
lousy
(16,)
football
(16,)
france
(16,)
excitement
(16,)
cost
(16,)
substance
(16,)
saturday
(16,)
losing
(16,)
talks
(16,)
priest
(16,)
destroy
(16,)
circumstances
(16,)
tedious
(16,)
training
(16,)
patrick
(16,)
thoughts
(16,)
hunt
(16,)

warn
(16,)
lion
(16,)
internet
(16,)
suspects
(16,)
overcome
(16,)
pool
(16,)
tour
(16,)
sin
(16,)
lessons
(16,)
credible
(16,)
wishes
(16,)
thousands
(16,)
2000
(16,)
advantage
(16,)
happily
(16,)
wealthy
(16,)
innocence
(16,)
tense
(16,)
catholic
(16,)
90's
(16,)
spin
(16,)
mindless
(16,)
racism
(16,)
broke
(16,)
miller
(16,)
bugs
(16,)
lucy
(16,)
cook
(16,)
guessing
(16,)
trial
(16,)
physically
(16,)
dubbing
(16,)
trite
(16,)
fish
(16,)
hundreds
(16,)
hood
(16,)
thrillers
(16,)
alike
(16,)
arm
(16,)
corner
(16,)
countries
(16,)
bag
(16,)
medical
(16,)
crisis
(16,)
fbi
(16,)
perform
(16,)
stranger
(16,)
pride
(16,)
dislike
(16,)
reed
(16,)
per
(16,)
controversial
(16,)
technique
(16,)
patient
(16,)
sexuality
(16,)
succeed
(16,)
wasting
(16,)
suffered
(16,)
enjoyment
(16,)
sings
(16,)
tied
(16,)
whereas
(16,)
separate
(16,)
lincoln
(16,)
uncomfortable
(16,)
gripping
(16,)
holmes
(16,)
ourselves
(16,)
hundred
(16,)
noble
(16,)
glimpse
(16,)
shines
(16,)
franchise
(16,)
ensemble
(16,)
r

...

keaton's
(16,)
settled
(16,)
gypo
(16,)
kelly's
(16,)
salesman
(16,)
mcqueen
(16,)
deanna
(16,)
clive
(16,)
verbal
(16,)
pathos
(16,)
cigarette
(16,)
cuban
(16,)
hoot
(16,)
instincts
(16,)
cohen
(16,)
possession
(16,)
meantime
(16,)
wheel
(16,)
satisfaction
(16,)
marilyn
(16,)
vomit
(16,)
expense
(16,)
duel
(16,)
shockingly
(16,)
deliciously
(16,)
update
(16,)
mysteriously
(16,)
pirates
(16,)
basket
(16,)
sources
(16,)
guinea
(16,)
cooking
(16,)
busey
(16,)
kathy
(16,)
zorro
(16,)
posing
(16,)
dumber
(16,)
nuances
(16,)
sunrise
(16,)
audience's
(16,)
jury
(16,)
newcomer
(16,)
encountered
(16,)
einstein
(16,)
attending
(16,)
fluid
(16,)
keys
(16,)
readily
(16,)
symbol
(16,)
phase
(16,)
pushes
(16,)
flavor
(16,)
franklin
(16,)
wash
(16,)
weary
(16,)
mortal
(16,)
flock
(16,)
trains
(16,)
brainless
(16,)
dynamics
(16,)
deputy
(16,)
defies
(16,)
ira
(16,)
policy
(16,)
controlling
(16,)
amidst
(16,)
skinny
(16,)
romances
(16,)
bachchan
(16,)
porter
(16,)
suggesting
(16,)
drab
(16,)
document


tones
(16,)
shocks
(16,)
similarity
(16,)
brett
(16,)
organization
(16,)
turtle
(16,)
villainous
(16,)
chaotic
(16,)
reserved
(16,)
26
(16,)
smell
(16,)
progressed
(16,)
inter
(16,)
interrupted
(16,)
arrogance
(16,)
fundamental
(16,)
monotonous
(16,)
ernie
(16,)
waking
(16,)
harrowing
(16,)
evokes
(16,)
trivial
(16,)
resume
(16,)
crushed
(16,)
chew
(16,)
manipulation
(16,)
rodriguez
(16,)
insert
(16,)
phoenix
(16,)
lynch's
(16,)
understatement
(16,)
wayans
(16,)
imagining
(16,)
jar
(16,)
blazing
(16,)
chewing
(16,)
chiba
(16,)
somethings
(16,)
bonham
(16,)
peril
(16,)
ripoff
(16,)
unnatural
(16,)
cocaine
(16,)
deemed
(16,)
collar
(16,)
slavery
(16,)
palm
(16,)
unhinged
(16,)
choir
(16,)
substitute
(16,)
mccarthy
(16,)
stiles
(16,)
bust
(16,)
lopez
(16,)
recover
(16,)
must've
(16,)
scheming
(16,)
finishing
(16,)
beers
(16,)
subsequently
(16,)
yell
(16,)
cheerful
(16,)
marcel
(16,)
men's
(16,)
anita
(16,)
python
(16,)
ninety
(16,)
achievements
(16,)
celebrities
(16,)
stumble
(16,)
aesthe

reese
(16,)
queens
(16,)
sustain
(16,)
shield
(16,)
goodman
(16,)
gibson
(16,)
morse
(16,)
booth
(16,)
cheech
(16,)
fabric
(16,)
overshadowed
(16,)
ewoks
(16,)
sensibilities
(16,)
1942
(16,)
maurice
(16,)
pee
(16,)
anguish
(16,)
spelling
(16,)
marrying
(16,)
stepmother
(16,)
lurid
(16,)
islands
(16,)
sophistication
(16,)
enthralling
(16,)
identified
(16,)
henchman
(16,)
peaks
(16,)
vet
(16,)
norris
(16,)
sandy
(16,)
kidnaps
(16,)
jewelry
(16,)
luc
(16,)
godard
(16,)
shout
(16,)
tina
(16,)
widower
(16,)
fixed
(16,)
cockney
(16,)
czech
(16,)
depend
(16,)
lampoon
(16,)
alba
(16,)
drawings
(16,)
geoffrey
(16,)
determine
(16,)
grandpa
(16,)
ceremony
(16,)
drastically
(16,)
sensational
(16,)
meadows
(16,)
russians
(16,)
supermarket
(16,)
allies
(16,)
theodore
(16,)
caution
(16,)
hairy
(16,)
nearest
(16,)
premiered
(16,)
hallucinations
(16,)
willingly
(16,)
cowardly
(16,)
distorted
(16,)
fiasco
(16,)
byron
(16,)
lars
(16,)
natali
(16,)
stares
(16,)
david's
(16,)
rant
(16,)
stricken
(16,)
just

(16,)
gee
(16,)
eastwood's
(16,)
peterson
(16,)
pertwee
(16,)
stretches
(16,)
opponents
(16,)
overwhelmed
(16,)
taker
(16,)
concludes
(16,)
melt
(16,)
custody
(16,)
hector
(16,)
agatha
(16,)
burden
(16,)
kamal
(16,)
crass
(16,)
academic
(16,)
marx
(16,)
shawn
(16,)
hopeful
(16,)
induced
(16,)
management
(16,)
dani
(16,)
mastermind
(16,)
plants
(16,)
wolves
(16,)
kiddie
(16,)
fable
(16,)
unlucky
(16,)
sematary
(16,)
boxes
(16,)
expanded
(16,)
booker
(16,)
theo
(16,)
michael's
(16,)
eddy
(16,)
kalifornia
(16,)
backyard
(16,)
azumi
(16,)
joanna
(16,)
condescending
(16,)
vault
(16,)
disappearing
(16,)
collecting
(16,)
mutated
(16,)
creepiness
(16,)
celine
(16,)
entertainer
(16,)
rehearsal
(16,)
nailed
(16,)
planted
(16,)
mold
(16,)
hoover
(16,)
woven
(16,)
revelations
(16,)
boyfriends
(16,)
premises
(16,)
january
(16,)
deathstalker
(16,)
choreographer
(16,)
lotr
(16,)
marlene
(16,)
slew
(16,)
contemplate
(16,)
archive
(16,)
discernible
(16,)
waving
(16,)
philadelphia
(16,)
nutty
(16,)
psyc

avant
(16,)
willy
(16,)
bum
(16,)
starving
(16,)
minions
(16,)
ounce
(16,)
integral
(16,)
64
(16,)
closeups
(16,)
texture
(16,)
smallest
(16,)
socialist
(16,)
charity
(16,)
ewan
(16,)
jeep
(16,)
ensuing
(16,)
indulge
(16,)
stud
(16,)
approved
(16,)
nuns
(16,)
decoration
(16,)
options
(16,)
blaine
(16,)
mo
(16,)
unsuccessful
(16,)
fernando
(16,)
spawned
(16,)
hickock
(16,)
aspirations
(16,)
capitalize
(16,)
murphy's
(16,)
employs
(16,)
grandparents
(16,)
amuse
(16,)
maniacal
(16,)
exposes
(16,)
insisted
(16,)
skimpy
(16,)
culminating
(16,)
it'd
(16,)
leia
(16,)
coincidences
(16,)
crowds
(16,)
overweight
(16,)
geeky
(16,)
answering
(16,)
olympia
(16,)
skipped
(16,)
researched
(16,)
projection
(16,)
insecure
(16,)
amateurs
(16,)
pursues
(16,)
munchies
(16,)
adored
(16,)
prevents
(16,)
weaves
(16,)
blaxploitation
(16,)
canon
(16,)
addresses
(16,)
praying
(16,)
jumbo
(16,)
turgid
(16,)
somber
(16,)
washing
(16,)
threats
(16,)
tasteful
(16,)
irs
(16,)
devastated
(16,)
tent
(16,)


In [33]:
sentence = "I really think this is amazing"

In [34]:
sequence = tokenizer.texts_to_sequences(sentence)

In [35]:
print(sequence)

[[11], [], [1478], [960], [4], [2008], [2008], [5123], [], [828], [2021], [11], [3355], [2290], [], [828], [2021], [11], [588], [], [11], [588], [], [4], [1978], [4], [4555], [11], [3355], [1329]]


In [36]:
sequence

[11]

In [43]:
reverse_word_index[4]

'a'